# Part 1 - SQL challenge

Jupyter Notebook and magic command `%%bigquery` were used to connect to the Bigquery ["COVID-19 Cases by Country" database](https://console.cloud.google.com/marketplace/product/european-cdc/covid-19-global-cases).
  
Here you can find details on how to configure Jupyter in order to connect to the Bigquery:
https://towardsdatascience.com/using-jupyter-notebook-to-manage-your-bigquery-analytics-c4dc7b2a4113

This notebook will use a json key as the authorization method. The exact path to the json key is defined in the .env file which is not located in the public repo.   
**Note:** If you want to clone a repo, you need to create the key yourself and save the path to it in your .env file.

In [1]:
# importing JSON_KEY_PATH variable from a local .env file

import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.getenv('JSON_KEY_PATH')

In [2]:
# Loading bigquery extension

%load_ext google.cloud.bigquery

Testing connection to Bigquery public database "COVID-19 Cases by Country" by simply quering first 5 records.

In [3]:
%%bigquery
SELECT * FROM bigquery-public-data.covid19_ecdc_eu.covid_19_geographic_distribution_worldwide
ORDER BY daily_deaths DESC
LIMIT 5 

Query is running:   0%|          |

Downloading:   0%|          |

,date,day,month,year,daily_confirmed_cases,daily_deaths,confirmed_cases,deaths,countries_and_territories,geo_id,country_territory_code,pop_data_2019
0,2020-04-16,16,4,2020,30148,4928,639664,30985,United_States_of_America,US,USA,329064917
1,2020-08-14,14,8,2020,9441,3935,507996,25648,Peru,PE,PER,32510462
2,2020-07-24,24,7,2020,4546,3887,371096,17654,Peru,PE,PER,32510462
3,2020-09-07,7,9,2020,-8261,3800,109784,10524,Ecuador,EC,ECU,17373657
4,2020-04-18,18,4,2020,30833,3770,702164,37054,United_States_of_America,US,USA,329064917


### Question 1
What time frame does the dataset cover? How many countries does it contain?

In [8]:
%%bigquery
SELECT MIN(date) AS min_date, MAX(date) AS max_date FROM bigquery-public-data.covid19_ecdc_eu.covid_19_geographic_distribution_worldwide

Query is running:   0%|          |

Downloading:   0%|          |

,min_date,max_date
0,2019-12-31,2020-12-14


**ANSWER:**  Data time range starts from 31-12-2019 to 14-12-2020.

During the brief data browsing, I noticed a strange name for one of the countries: *Cases_on_an_international_conveyance_Japan*

After a quick research, I found out that this name stands for all cases on the Diamond Princess cruise ship.

In [6]:
%%bigquery
SELECT DISTINCT(countries_and_territories) AS countries FROM bigquery-public-data.covid19_ecdc_eu.covid_19_geographic_distribution_worldwide

Query is running:   0%|          |

Downloading:   0%|          |

,countries
0,Cases_on_an_international_conveyance_Japan
1,Afghanistan
2,United_Arab_Emirates
3,Armenia
4,Australia
...,...
209,United_Republic_of_Tanzania
210,United_States_Virgin_Islands
211,Democratic_Republic_of_the_Congo
212,Saint_Vincent_and_the_Grenadines


In [7]:
%%bigquery
SELECT COUNT(DISTINCT(countries_and_territories)) AS countries FROM bigquery-public-data.covid19_ecdc_eu.covid_19_geographic_distribution_worldwide

Query is running:   0%|          |

Downloading:   0%|          |

,countries
0,214


**ANSWER:** Assuming that we treat all unique names in the `countries_and_territories` column as countries, there are 213 countries in the dataset (214 - 1 ship = 213)

### Question 2

Which country had the most deaths due to Covid-19 on a single day, and which day was that?

In [9]:
%%bigquery
SELECT date, daily_deaths, countries_and_territories FROM bigquery-public-data.covid19_ecdc_eu.covid_19_geographic_distribution_worldwide
ORDER BY daily_deaths DESC
LIMIT 1 

Query is running:   0%|          |

Downloading:   0%|          |

,date,daily_deaths,countries_and_territories
0,2020-04-16,4928,United_States_of_America


**ANSWER:** On a single day, the most deaths (4928) were in the USA on 16-04-2020. 

### Question 3 

Which country had the most deaths due to Covid-19 in August 2020, and how many?

In [12]:
%%bigquery
SELECT countries_and_territories AS country, SUM(daily_deaths) AS deaths_in_august FROM bigquery-public-data.covid19_ecdc_eu.covid_19_geographic_distribution_worldwide
GROUP BY country, month
HAVING month = 8
ORDER BY deaths_in_august DESC
LIMIT 5

Query is running:   0%|          |

Downloading:   0%|          |

,country,deaths_in_august
0,United_States_of_America,30999
1,Brazil,29565
2,India,28722
3,Mexico,18158
4,Peru,9767


**ANSWER:** In August 2020, the most deaths due to COVID-19 were in the USA - 30 999 cases.

### Question 4 

How many countries have complete (i.e. one sample per day) data from at least April 2020 to (including) November 2020?

There are exactly 244 days in the period from 2020-04-01 to 2020-11-30. This information was used in the subquery to select only those records that meet this criterion, i.e. have one sample each day during this period.

In [13]:
%%bigquery
SELECT COUNT(*) AS countries_with_full_record FROM 
(SELECT country_territory_code FROM 
(SELECT * FROM bigquery-public-data.covid19_ecdc_eu.covid_19_geographic_distribution_worldwide
WHERE month >= 4 AND month <=11)
GROUP BY country_territory_code
HAVING COUNT(DISTINCT(date)) = 244)

Query is running:   0%|          |

Downloading:   0%|          |

,countries_with_full_record
0,196


**ANSWER:** 196 countries have full data (one sample per day) from April (2020-04-01) to November (2020-11-30)

### Question 5 

On which day and in which country was Covid-19 deadliest, defined as `daily_deaths / sum of daily confirmed cases over last 14 days`. Use only the subset of the countries found in Question 4 to answer this question. What do you observe?

Using the following query, the 15 records with the highest death ratio are displayed in descending order. The window function was used to calculate the death ratio. If the sum of the last 14 days is zero, then the death ratio is set to 0.  
In addition to the death ratio itself, other data is displayed, such as the sum of cases from the last 14 days, the number of deaths per day, or the number of confirmed cases per day to give the results a broader context.

In [17]:
%%bigquery
SELECT date, countries_and_territories, 
CASE
    WHEN SUM(daily_confirmed_cases) OVER (PARTITION BY country_territory_code ORDER BY date ROWS BETWEEN 13 PRECEDING AND CURRENT ROW) = 0 THEN NULL
    ELSE daily_deaths / NULLIF(SUM(daily_confirmed_cases) OVER (PARTITION BY country_territory_code ORDER BY date ROWS BETWEEN 13 PRECEDING AND CURRENT ROW), 0)
END AS death_ratio,
SUM(daily_confirmed_cases) OVER (PARTITION BY country_territory_code ORDER BY date ROWS BETWEEN 13 PRECEDING AND CURRENT ROW) AS sum_confirmed_cases_last_14_days,
daily_confirmed_cases, daily_deaths, confirmed_cases, deaths
FROM bigquery-public-data.covid19_ecdc_eu.covid_19_geographic_distribution_worldwide
WHERE country_territory_code IN (SELECT country_territory_code FROM
(SELECT * FROM bigquery-public-data.covid19_ecdc_eu.covid_19_geographic_distribution_worldwide
WHERE month >= 4 AND month <=11)
GROUP BY country_territory_code
HAVING COUNT(*) = 244)
ORDER BY death_ratio DESC
LIMIT 15

Query is running:   0%|          |

Downloading:   0%|          |

,date,countries_and_territories,death_ratio,sum_confirmed_cases_last_14_days,daily_confirmed_cases,daily_deaths,confirmed_cases,deaths
0,2020-09-07,Ecuador,1.885856,2015,-8261,3800,109784,10524
1,2020-04-17,China,1.000776,1289,352,1290,83754,4636
2,2020-05-08,Aruba,1.000000,1,0,1,101,3
3,2020-03-15,Sudan,1.000000,1,0,1,1,1
4,2020-08-01,Fiji,1.000000,1,0,1,27,1
5,2020-04-20,British_Virgin_Islands,1.000000,1,1,1,4,1
6,2020-03-16,Guatemala,1.000000,1,0,1,1,1
7,2020-03-13,Guyana,1.000000,1,1,1,1,1
8,2020-08-29,Fiji,1.000000,1,0,1,28,2
9,2020-02-20,Iran,1.000000,2,2,2,2,2


**ANSWER:** Answering directly the question asked, based on the proposed death ratio method, the deadliest day in terms of COVID-19 was 07-09-2020 in Ecuador with a death ratio of: 1.885856  

However, looking at other data around this day, it can be said that the death ratio is not reliable. Below is a table with records showing the course of the pandemic in Ecuador during this period. It can be immediately noticed that the number of deaths (3800) on 2020-09-07 is an outlier and strongly differs from the number of deaths in the adjacent days. The number of confirmed cases is also negative, which automatically increases the death rate.

In [25]:
%%bigquery
SELECT countries_and_territories AS country, date, daily_confirmed_cases, daily_deaths, confirmed_cases, deaths FROM bigquery-public-data.covid19_ecdc_eu.covid_19_geographic_distribution_worldwide
WHERE date BETWEEN '2020-08-24' AND '2020-09-10'
AND countries_and_territories = 'Ecuador'
ORDER BY date DESC

Query is running:   0%|          |

Downloading:   0%|          |

,country,date,daily_confirmed_cases,daily_deaths,confirmed_cases,deaths
0,Ecuador,2020-09-10,1409,74,112166,10701
1,Ecuador,2020-09-09,665,51,110757,10627
2,Ecuador,2020-09-08,308,52,110092,10576
3,Ecuador,2020-09-07,-8261,3800,109784,10524
4,Ecuador,2020-09-06,870,50,118045,6724
5,Ecuador,2020-09-05,815,26,117175,6674
6,Ecuador,2020-09-04,903,29,116360,6648
7,Ecuador,2020-09-03,1148,48,115457,6619
8,Ecuador,2020-09-02,542,15,114309,6571
9,Ecuador,2020-09-01,119,1,113767,6556


Among the 15 countries with the highest death ratio, a similar situation is in the case of China and Sudan:

In [26]:
# CHINA

%%bigquery
SELECT countries_and_territories AS country, date, daily_confirmed_cases, daily_deaths, confirmed_cases, deaths FROM bigquery-public-data.covid19_ecdc_eu.covid_19_geographic_distribution_worldwide
WHERE date BETWEEN '2020-04-03' AND '2020-04-20'
AND countries_and_territories = 'China'
ORDER BY date DESC

Query is running:   0%|          |

Downloading:   0%|          |

,country,date,daily_confirmed_cases,daily_deaths,confirmed_cases,deaths
0,China,2020-04-20,14,0,83817,4636
1,China,2020-04-19,18,0,83803,4636
2,China,2020-04-18,31,0,83785,4636
3,China,2020-04-17,352,1290,83754,4636
4,China,2020-04-16,50,0,83402,3346
5,China,2020-04-15,49,1,83352,3346
6,China,2020-04-14,94,0,83303,3345
7,China,2020-04-13,112,2,83209,3345
8,China,2020-04-12,93,0,83097,3343
9,China,2020-04-11,79,3,83004,3343


In [27]:
# SUDAN

%%bigquery
SELECT countries_and_territories AS country, date, daily_confirmed_cases, daily_deaths, confirmed_cases, deaths FROM bigquery-public-data.covid19_ecdc_eu.covid_19_geographic_distribution_worldwide
WHERE date BETWEEN '2020-10-26' AND '2020-11-12'
AND countries_and_territories = 'Sudan'
ORDER BY date DESC

Query is running:   0%|          |

Downloading:   0%|          |

,country,date,daily_confirmed_cases,daily_deaths,confirmed_cases,deaths
0,Sudan,2020-11-12,191,0,14346,1116
1,Sudan,2020-11-11,0,0,14155,1116
2,Sudan,2020-11-10,65,1,14155,1116
3,Sudan,2020-11-09,147,278,14090,1115
4,Sudan,2020-11-08,0,0,13943,837
5,Sudan,2020-11-07,0,0,13943,837
6,Sudan,2020-11-06,0,0,13943,837
7,Sudan,2020-11-05,77,0,13943,837
8,Sudan,2020-11-04,47,0,13866,837
9,Sudan,2020-11-03,0,0,13819,837


The rest of the cases are when there are 1-2 deaths for 1-2 cases in the last 14 days, which results in a very high death ratio, but it doesn't look like the deadliest day of the pandemic (eg. comparing to the maximum number of deaths in one day).

**CONCLUSION:** The death ratio for this dataset is not very reliable because it is prone to outliers. Also, that ratio will take on high values when there are single cases. Therefore, to analyze the level of mortality due to COVID-19, it would be necessary to deal with outliers and also take into account the maximum number of deaths on a given day.